In [1]:
import sys
import re
import os
import numpy as np
import pandas as pd
import pysam
import matplotlib.pyplot as plt
import seaborn as sns
import pysam
import gget
import mappy as mp
from Bio.Blast import NCBIWWW

In [2]:
n_sample = 10
n_rows = 5000
root_dir = "/nfs/turbo/umms-indikar/shared/projects/poreC/data/fastq/p2/"

res = []

def read_fastq(fpath, n_rows):
    """a function to parse a fastq file """
    count = -1
    res = []
    for read in pysam.FastxFile(fpath):
        count += 1
        if count == n_rows:
            break

        NlaIII = read.sequence.count('CATG')
        quals = read.get_quality_array()
        row = {
            'read_name' : read.name,
            'seq_length' : len(read.sequence),
            'n_NlaIII' : NlaIII,
            'mean_base_quality' : np.mean(quals),
            'median_base_quality' : np.median(quals),
            'min_base_quality' : np.min(quals),
            'max_base_quality' : np.max(quals),
            'sequence' : read.sequence,
        }

        res.append(row)
    return pd.DataFrame(res)
    
res = []
for run in os.listdir(root_dir):

    subdir_path = f"{root_dir}{run}/merged_fastq/"
    sample_files = np.random.choice(os.listdir(subdir_path), n_sample)

    for sample in sample_files:
        sample_path = f"{subdir_path}{sample}"

        df = read_fastq(sample_path, n_rows)
        df['run'] = run
        df['cell'] = sample
        res.append(df)

res = pd.concat(res)
res.head()

,read_name,seq_length,n_NlaIII,mean_base_quality,median_base_quality,min_base_quality,max_base_quality,sequence,run,cell
0,eecfa339-c722-4d79-979c-fdc347958902,619,1,15.381260,17.0,2,26,ATGTTAACATTTGCCCCTTCGTTCAGTTACGTATTGCTGGTGCTGT...,scPC96o_3,barcode87.fastq
1,c24e4f03-296c-4ffc-a634-1b6f5f2e0d99,570,1,32.808772,37.0,3,50,GGTGTTATGTGGTGTGCTTGGTTCAGTTACGTATTGCTGGTGCTGT...,scPC96o_3,barcode87.fastq
2,4758f7da-1a21-4e15-83f2-dfc1e71446be,277,1,29.490975,36.0,2,50,GTGCATCTGCTTGGTTCGATTACGTATTGCTGGTGCTGTACATGCT...,scPC96o_3,barcode87.fastq
3,ba527fe5-f141-442e-b6e9-3b9a95713d95,809,1,19.383189,20.0,1,50,ATGATCTCGTTCAGTTACGTATTGCTGTTGGTGCTGATATTGCGAG...,scPC96o_3,barcode87.fastq
4,503f506f-3cb9-4d9f-af97-361fd92d72ae,568,1,17.876761,21.0,2,33,ACTGTCCTGTACTTGGTTCAGCTACCGTATTGCTGGAAAGTACTCT...,scPC96o_3,barcode87.fastq
